#### Import the aima-python repo and any necessary libraries

In [1]:
import sys
import os

# Add the path to the aima-python repo to sys.path
sys.path.append(os.path.abspath('./aima_python'))

from aima_python.games4e import Game
from aima_python.utils4e import vector_add, MCT_Node, ucb
from collections import defaultdict

import random
random.seed(109)

#### Implement our verison of Mancala by subclassing the Game class from aima

In [2]:
class Mancala(Game):
    """Mancala game implementation."""

    def __init__(self):
        self.squares = {i for i in range(14) if i != 6 and i != 13}  # Indices of pits (excluding mancalas)
        self.initial = Board(pits=[4] * 6 + [0] + [4] * 6 + [0], to_move='P1')

    def actions(self, board):
        """Return a list of legal moves (non-empty pits on the current player's side)."""
        start, end = (0, 6) if board.to_move == 'P1' else (7, 13)
        return [i for i in range(start, end) if board.pits[i] > 0]

    def result(self, board, action):
        """Apply the move (distributing stones) and return the new board state."""
        pits = board.pits[:]
        player = board.to_move
        stones = pits[action]
        pits[action] = 0

        idx = action
        while stones > 0:
            idx = (idx + 1) % 14
            if (player == 'P1' and idx == 13) or (player == 'P2' and idx == 6):
                continue  # Skip opponent's mancala
            pits[idx] += 1
            stones -= 1

        # Capture condition
        if (player == 'P1' and 0 <= idx < 6 or player == 'P2' and 7 <= idx < 13) \
                and pits[idx] == 1 and pits[12 - idx] > 0:
            pits[6 if player == 'P1' else 13] += pits[idx] + pits[12 - idx]
            pits[idx] = pits[12 - idx] = 0

        # Determine next player
        next_to_move = player if (player == 'P1' and idx == 6 or player == 'P2' and idx == 13) else ('P2' if player == 'P1' else 'P1')

        return board.new({'pits': pits, 'to_move': next_to_move})

    def utility(self, board, player):
        """Return the game utility for the given player."""
        if self.is_terminal(board): # Calculate the utility function only at terminal nodes
            score_p1 = sum(board.pits[:7])  # P1's Mancala score
            score_p2 = sum(board.pits[7:])  # P2's Mancala score
            if player == 'P1':
                return score_p1 - score_p2  # Max - Min for P1
            else:
                return score_p2 - score_p1  # Max - Min for P2
            return 0

    def is_terminal(self, board):
        """Check if the game has ended."""
        return all(p == 0 for p in board.pits[:6]) or all(p == 0 for p in board.pits[7:13])

    def display(self, board, current_player=None, move_from=None):
        """Display the board state."""
        
        print("  " + " ".join(map(str, board.pits[12:6:-1])))
        print(f"{board.pits[13]}                  {board.pits[6]}")
        print("  " + " ".join(map(str, board.pits[:6])))
        print("\n")

        if current_player is not None and move_from is not None:
            print(f"Player {current_player} moved from pit {move_from}")

#### Implement our version of the mancala board

In [3]:
class Board(defaultdict):
    """A Mancala board with pits and a player to move."""

    def __init__(self, pits=None, to_move=None, **kwds):
        super().__init__(int)
        self.pits = pits or [0] * 14
        self.to_move = to_move

    def new(self, changes: dict, **kwds) -> 'Board':
        """Create a new board state with the specified changes."""
        board = Board(pits=self.pits[:], to_move=self.to_move, **kwds)
        board.__dict__.update(changes)
        return board

    def __hash__(self):
        return hash(tuple(self.pits)) + hash(self.to_move)

    def __repr__(self):
        return f"Mancala({self.pits}, {self.to_move})"

#### Create a random player and a play game function to be able to play games

In [4]:
# Example game simulation
from random import choice

def random_player(game, board):
    legal_moves = game.actions(board)
    return choice(legal_moves)

def play_game(game, players, verbose=False):
    """Simulate a game between two players"""
    state = game.initial
    while not game.is_terminal(state):
        current_player = players[state.to_move]  # Get the current player
        action = current_player(game, state)  # Get the player's move
        state = game.result(state, action)  # Apply the move and get the new state
        
        if verbose:
            game.display(state)  # Display the board state if verbose is True
    
    return game.utility(state, 'P1')  # Return the utility of the game for player 'P1'

#### Use this function to set what type of player is playing

In [5]:
def player(search_algorithm):
    """A game player who uses the specified search algorithm."""
    return lambda game, state: search_algorithm(game, state)

#### Use the simulate game function in order to simulate numerous games
#### We need to update this so we can choose the minimax or alphabeta players

In [6]:
def simulate_game(player1, player2, start_with_p1=True):
    """Simulate a single game of Mancala between specified players."""
    game = Mancala()
    board = game.initial.new({'to_move': 'P1' if start_with_p1 else 'P2'})
    moves = 0
    while not game.is_terminal(board):
        current_player = player1 if board.to_move == 'P1' else player2
        action = current_player(game, board)
        board = game.result(board, action)
        moves += 1
    winner = 'P1' if sum(board.pits[:7]) > sum(board.pits[7:]) else 'P2'
    return winner, moves

In [7]:
def simulate_multiple_games(player1, player2, num_games=100):
    """Simulate multiple games and track win percentages and average moves."""
    p1_wins, p2_wins, total_moves = 0, 0, 0
    for i in range(num_games):
        start_with_p1 = (i % 2 == 0)  # Alternate starting players
        winner, moves = simulate_game(player1, player2, start_with_p1=start_with_p1)
        if winner == 'P1':
            p1_wins += 1
        else:
            p2_wins += 1
        total_moves += moves
    return (p1_wins / num_games) * 100, (p2_wins / num_games) * 100, total_moves / num_games

In [8]:
def minimax(game, state, max_depth=5):
    def max_value(state, depth):
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state)
        v = -float('inf')
        for action in game.actions(state):
            v = max(v, min_value(game.result(state, action), depth - 1))
        return v

    def min_value(state, depth):
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state)
        v = float('inf')
        for action in game.actions(state):
            v = min(v, max_value(game.result(state, action), depth - 1))
        return v

    # The minimax search returns the best action for the player whose turn it is.
    best_action = None
    best_value = -float('inf') if state.to_move == 'P1' else float('inf')
    
    for action in game.actions(state):
        if state.to_move == 'P1':
            value = min_value(game.result(state, action), max_depth - 1)
            if value > best_value:
                best_value, best_action = value, action
        else:
            value = max_value(game.result(state, action), max_depth - 1)
            if value < best_value:
                best_value, best_action = value, action

    return best_action

def heuristic_evaluation(state):
    """Evaluate the board state with a sophisticated heuristic."""
    p1_mancala = state.pits[6]
    p2_mancala = state.pits[13]
    stones_on_board_p1 = sum(state.pits[:6])
    stones_on_board_p2 = sum(state.pits[7:13])

    # Mancala advantage
    mancala_score = p1_mancala - p2_mancala

    # Board control: prioritize having more stones on your side
    board_control = stones_on_board_p1 - stones_on_board_p2

    # Extra-turn potential (favoring empty pits with stones opposite)
    extra_turns = sum(1 for i in range(6) if state.pits[i] == 0 and state.pits[12 - i] > 0)

    return mancala_score + 0.5 * board_control + 0.3 * extra_turns

In [9]:
def alphabeta(game, state, alpha=-float('inf'), beta=float('inf'), max_depth=5):
    """Alpha-beta pruning algorithm with a depth limit."""
    def max_value(state, alpha, beta, depth):
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state), None
        v, best_action = -float('inf'), None
        for action in game.actions(state):
            v2, _ = min_value(game.result(state, action), alpha, beta, depth - 1)
            if v2 > v:
                v, best_action = v2, action
            if v >= beta:
                return v, best_action
            alpha = max(alpha, v)
        return v, best_action

    def min_value(state, alpha, beta, depth):
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state), None
        v, best_action = float('inf'), None
        for action in game.actions(state):
            v2, _ = max_value(game.result(state, action), alpha, beta, depth - 1)
            if v2 < v:
                v, best_action = v2, action
            if v <= alpha:
                return v, best_action
            beta = min(beta, v)
        return v, best_action

    return max_value(state, alpha, beta, max_depth)[1]


In [10]:
minimax_player = player(lambda game, state: minimax(game, state))
alphabeta_player = player(lambda game, state: alphabeta(game, state))

In [13]:
p1_win_percentage, p2_win_percentage, avg_moves = simulate_multiple_games(minimax_player, random_player, num_games=100)
print("Minimax Player vs Random Player")
print(f"Player 1 win percentage: {p1_win_percentage}%")
print(f"Player 2 win percentage: {p2_win_percentage}%")
print(f"Average number of moves: {avg_moves}")

Minimax Player vs Random Player
Player 1 win percentage: 97.0%
Player 2 win percentage: 3.0%
Average number of moves: 34.835


In [14]:
p1_win_percentage, p2_win_percentage, avg_moves = simulate_multiple_games(alphabeta_player, random_player, num_games=200)
print("AlphaBeta Player vs Random Player")
print(f"Player 1 win percentage: {p1_win_percentage}%")
print(f"Player 2 win percentage: {p2_win_percentage}%")
print(f"Average number of moves: {avg_moves}")

AlphaBeta Player vs Random Player
Player 1 win percentage: 98.0%
Player 2 win percentage: 2.0%
Average number of moves: 34.455
